> 코루틴을 이용하면 파이썬 프로그램에서 스레드를 여러개 생성 하지 않고, 동시에 많은 함수를 실행하는 것처럼 보이게 할 수 있다.
> 코루틴은 제너레이트롤 소비하는 코드에서 send 함수를 사용하여 역으로 제너레이터 함수의 각 yield 표현식에 값을 보낼 수 있게 하는 방법으로 동작한다.

In [29]:
def my_coroutine():
    
    while True:
        received = yield
        print('Received: ', received)
        
it = my_coroutine()
next(it)

In [30]:
it.send(10)
it.send(20)

Received:  10
Received:  20


In [88]:
from collections import namedtuple

Query = namedtuple('Query', ('y', 'x'))
Transition = namedtuple('Transition', ('y', 'x', 'state'))

In [89]:
ALIVE = '*'
EMPTY = '-'

def count_neighbors(y, x):
    n_ = yield Query(y + 1, x + 0)
    ne = yield Query(y + 1, x + 1)
    e_ = yield Query(y, x + 1)
    se = yield Query(y - 1, x + 1)
    s_ = yield Query(y - 1, x)
    sw = yield Query(y - 1, x - 1)
    w_ = yield Query(y, x - 1)
    nw = yield Query(y + 1, x - 1)
    
    neighbor_states = [n_, ne, e_, se, s_, sw, w_, nw]
    count = 0
    for state in neighbor_states:
        if state == ALIVE:
            count += 1
    return count

In [90]:
def game_logic(state, neighbors):
    if state == ALIVE:
        if neighbors < 2:
            return EMPTY
        elif neighbors > 3:
            return EMPTY
    else:
        if neighbors == 3:
            return ALIVE
    return state

In [91]:
def step_cell(y, x):
    state = yield Query(y, x)
    neighbors = yield from count_neighbors(y, x)
    next_state = game_logic(state, neighbors)
    yield Transition(y, x, next_state)

In [92]:
TICK = object()
def simulate(height, width):
    while True:
        for y in range(height):
            for x in range(width):
                yield from step_cell(y, x)
        yield TICK

In [93]:
class Grid(object):
    def __init__(self, height, width):
        self.height = height
        self.width = width
        self.rows = []
        for _ in range(self.height):
            self.rows.append([EMPTY] * self.width)
    
    def __str__(self):
        return '\n'.join([''.join(row) for row in self.rows])
    
    def query(self, y, x):
        return self.rows[y % self.height][x % self.width]
    
    def assign(self, y, x, state):
        self.rows[y % self.height][x % self.width] = state

In [97]:
def live_a_generation(grid, sim):
    progeny = Grid(grid.height, grid.width)
    item = next(sim)
    while item is not TICK:
        if isinstance(item, Query):
            state = grid.query(item.y, item.x)
            item = sim.send(state)
        else:
            progeny.assign(item.y, item.x, item.state)
            item = next(sim)
    return progeny


In [98]:
class ColumnPrinter(object):
    def __init__(self):
        self.columns = []
    
    def append(self, data):
        self.columns.append(data)
    
    def __str__(self):
        row_count = 1
        for data in self.columns:
            row_count = max(row_count, len(data.splitlines()) + 1)
        rows = [''] * row_count
        for j in range(row_count):
            for i, data in enumerate(self.columns):
                line = data.splitlines()[max(0, j - 1)]
                if j == 0:
                    padding = ' ' * (len(line) // 2)
                    rows[j] += padding + str(i) + padding
                else:
                    rows[j] += line
                if (i + 1) < len(self.columns):
                    rows[j] += '|'
        return '\n'.join(rows)

In [100]:
grid = Grid(5, 9)
grid.assign(0, 3, ALIVE)
grid.assign(1, 4, ALIVE)
grid.assign(2, 2, ALIVE)
grid.assign(2, 3, ALIVE)
grid.assign(2, 4, ALIVE)

columns = ColumnPrinter()
sim  = simulate(grid.height, grid.width)
for i in range(5):
    columns.append(str(grid))
    grid = live_a_generation(grid, sim)

print(columns)

    0    |    1    |    2    |    3    |    4    |    5    |    6    |    7    |    8    |    9    
---*-----|---------|---------|---------|---------|---------|---------|---------|---------|-----*---
----*----|--*-*----|----*----|---*-----|----*----|---------|---------|---------|---------|---------
--***----|---**----|--*-*----|----**---|-----*---|---*-*---|-----*---|----*----|-----*---|---------
---------|---*-----|---**----|---**----|---***---|----**---|---*-*---|-----**--|------*--|----*-*--
---------|---------|---------|---------|---------|----*----|----**---|----**---|----***--|-----**--
